# Consultas con joins
### **Ingeniería de datos**
**Profesor: Domagoj Vrgoč**

### Introducción

Cruces, o joins, son la operación más importante en SQL, porque nos permiten mezclar datos de distintas tablas. En este tutorial practicaremos el uso de joins para analizar un caso práctico, y repsonder consultas naturales ene este caso.

### Requisitos

Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar *Google colab* (https://colab.research.google.com), que es un entorno virtual permitiendo armar un servidor de bases de datos, y conectarse con este servidor. Para la conexión ocuparemos la herramienta llamada Jupyter Notebooks. Esta herramienta permite conectarse con un servidor SQL de la misma manera cómo hacerlo a través de la consola en un servidor local. 

El motor de bases de datos que ocuparemos en este curso se llama PostgreSQL, y uno siempre puede instalarlo localmente en su computador. Idea de ocupar Google colab es saltarse este paso, y no tener problemas con instalar, habilitar, o correr un motor de bases de datos.

Por lo tanto, para una actividad de SQL, en este curso siempre ocuparemos Jupyter Notebooks con Google colab. Para esto, se les entregará un archivo con extensión .ipynb, cual hay que subir a la plataforma Google Colab. Al inicio del tutorial mostraremos cómo funciona este proceso.


### Esquema

Para esta actividad vamos a trabajar con el siguiente esquema:

- `Peliculas(pid, pnombre, paño, pcategoria, pcalificacion, pdirector)`

- `Actores(aid, anombre, aedad)`

- `actuo_en(aid, pid, rol)`

Que corresponde a películas, actores, y la información de los roles interpretados por una actor en una película.

Las llaves en nuestro caso son:
1. `pid`, para `Peliculas`
2. `aid` para `Actores`
3. `(aid,pid,rol)` para `actuo_en`.

Es importante notar que en este caso `rol` forma parte de la llave primaria de la relación `actuo_en`, dado que debemos modelar el caso donde un actor juega dos roles distintos en una misma película.

## Tutorial

Lo primero que hay que hacer es subir este notebook a https://colab.research.google.com

### Iniciar el servidor

Para iniciar el servidor virtual, *instalar* la base de datos postgres debe correr el siguiente bloque:

In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [748 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

### Creando la base de datos

Para crear y poblar nuestra base de datos, corramo el siguiente bloque de código:

In [ ]:
%%sql

DROP TABLE IF EXISTS Peliculas;
DROP TABLE IF EXISTS Actores;
DROP TABLE IF EXISTS Actuo_En;

CREATE TABLE IF NOT EXISTS Peliculas(
    pid int PRIMARY KEY,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);

CREATE TABLE Actores(
    aid int PRIMARY KEY,
    anombre varchar(30),
    aedad int
);

CREATE TABLE actuo_en(
    aid int,
    pid int,
    rol varchar(30),
    PRIMARY KEY (aid,pid,rol)
);

INSERT INTO Peliculas VALUES(1,'Avengers:Endgame',2019,'SciFi',8.4,'Brothers Russo');
INSERT INTO Peliculas VALUES(2,'Captain America: Civil War',2016,'SciFi',8.7,'Brothers Russo');
INSERT INTO Peliculas VALUES(3,'Iron Man',2008,'SciFi',9.1,'John Favreu');
INSERT INTO Peliculas VALUES(4,'Batman: The Dark Knight',2008,'Thriller',8.3,'Christoper Nolan');
INSERT INTO Peliculas VALUES(5,'Batman: The Dark Knight Rises',2012,'Thriller',7.9,'Christoper Nolan');
INSERT INTO Peliculas VALUES(6,'Interstellar',2014,'Drama',8.4,'Christoper Nolan');
INSERT INTO Peliculas VALUES(7,'Sherlock Holmes',2009,'Mystery',7.8,'Guy Ritchie');
INSERT INTO Peliculas VALUES(8,'Avengers: Age of Ultron',2015,'SciFi',8.3,'Joss Whedon');
INSERT INTO Peliculas VALUES(9,'Doctor Strange',2016,'SciFi',8.8,'Scott Derrickson');

INSERT INTO Actores VALUES(1,'Robert Downey Jr.',57);
INSERT INTO Actores VALUES(2,'Scarlett Johansson',37);
INSERT INTO Actores VALUES(3,'Chris Evans',40);
INSERT INTO Actores VALUES(4,'Christian Bale',48);
INSERT INTO Actores VALUES(5,'Anne Hathaway',39);
INSERT INTO Actores VALUES(6,'Paul Bettany',50);
INSERT INTO Actores VALUES(7,'Benedict Cumberbatch',45);

INSERT INTO actuo_en VALUES(1,1,'Tony Stark');
INSERT INTO actuo_en VALUES(1,2,'Tony Stark');
INSERT INTO actuo_en VALUES(1,3,'Tony Stark');
INSERT INTO actuo_en VALUES(1,7,'Sherlock Holmes');
INSERT INTO actuo_en VALUES(2,1,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(2,2,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(3,1,'Steve Rogers');
INSERT INTO actuo_en VALUES(3,2,'Steve Rogers');
INSERT INTO actuo_en VALUES(4,4,'Bruce Wayne');
INSERT INTO actuo_en VALUES(4,5,'Bruce Wayne');
INSERT INTO actuo_en VALUES(5,5,'Selina Kyle');
INSERT INTO actuo_en VALUES(5,6,'Amelia Brand');
INSERT INTO actuo_en VALUES(6,8,'J.A.R.V.I.S.');
INSERT INTO actuo_en VALUES(6,8,'Vision');
INSERT INTO actuo_en VALUES(7,9,'Doctor Strange');
INSERT INTO actuo_en VALUES(7,9,'Dormammu');
INSERT INTO actuo_en VALUES(1,8,'Tony Stark');
INSERT INTO actuo_en VALUES(3,8,'Steve Rogers');
INSERT INTO actuo_en VALUES(2,8,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(6,3,'J.A.R.V.I.S.');


 * postgresql+psycopg2://@/postgres


""


Ahora podemos visualizar el contenido de las tablas:

In [ ]:
%%sql

SELECT * FROM Peliculas;

 * postgresql+psycopg2://@/postgres


,pid,pnombre,paño,pcategoria,pcalificacion,pdirector
0,1,Avengers:Endgame,2019,SciFi,8.4,Brothers Russo
1,2,Captain America: Civil War,2016,SciFi,8.7,Brothers Russo
2,3,Iron Man,2008,SciFi,9.1,John Favreu
3,4,Batman: The Dark Knight,2008,Thriller,8.3,Christoper Nolan
4,5,Batman: The Dark Knight Rises,2012,Thriller,7.9,Christoper Nolan
5,6,Interstellar,2014,Drama,8.4,Christoper Nolan
6,7,Sherlock Holmes,2009,Mystery,7.8,Guy Ritchie
7,8,Avengers: Age of Ultron,2015,SciFi,8.3,Joss Whedon
8,9,Doctor Strange,2016,SciFi,8.8,Scott Derrickson


In [ ]:
%%sql

SELECT * FROM Actores;

 * postgresql+psycopg2://@/postgres


,aid,anombre,aedad
0,1,Robert Downey Jr.,57
1,2,Scarlett Johansson,37
2,3,Chris Evans,40
3,4,Christian Bale,48
4,5,Anne Hathaway,39
5,6,Paul Bettany,50
6,7,Benedict Cumberbatch,45


In [ ]:
%%sql

SELECT * FROM actuo_en;

 * postgresql+psycopg2://@/postgres


,aid,pid,rol
0,1,1,Tony Stark
1,1,2,Tony Stark
2,1,3,Tony Stark
3,1,7,Sherlock Holmes
4,2,1,Natasha Romanoff
5,2,2,Natasha Romanoff
6,3,1,Steve Rogers
7,3,2,Steve Rogers
8,4,4,Bruce Wayne
9,4,5,Bruce Wayne


### Analizando la base de datos con joins

Ahora que conocemos a nuestros datos, practicaremos el uso de joins para analizarlos en más detalles.

### Pregunta 1

Para partir, queremos devolver el nombre de cada actor, junto con el nombre de todos los personajes que interpretó este actor. La consulta para realizar esto es la siguiente:

In [ ]:
%%sql

SELECT Actores.anombre, actuo_en.rol
FROM Actores, actuo_en
WHERE Actores.aid = actuo_en.aid


 * postgresql+psycopg2://@/postgres


,anombre,rol
0,Robert Downey Jr.,Tony Stark
1,Robert Downey Jr.,Tony Stark
2,Robert Downey Jr.,Tony Stark
3,Robert Downey Jr.,Sherlock Holmes
4,Scarlett Johansson,Natasha Romanoff
5,Scarlett Johansson,Natasha Romanoff
6,Chris Evans,Steve Rogers
7,Chris Evans,Steve Rogers
8,Christian Bale,Bruce Wayne
9,Christian Bale,Bruce Wayne


En esta lista podemos observar que hay muchas repeticiones. Para eliminarlas, podemos ocupar `DISTINCT`. Adicionalmente, podemos ordenar por actor:

In [ ]:
%%sql

SELECT DISTINCT Actores.anombre, actuo_en.rol
FROM Actores, actuo_en
WHERE Actores.aid = actuo_en.aid
ORDER BY Actores.anombre

 * postgresql+psycopg2://@/postgres


,anombre,rol
0,Anne Hathaway,Amelia Brand
1,Anne Hathaway,Selina Kyle
2,Benedict Cumberbatch,Doctor Strange
3,Benedict Cumberbatch,Dormammu
4,Chris Evans,Steve Rogers
5,Christian Bale,Bruce Wayne
6,Paul Bettany,J.A.R.V.I.S.
7,Paul Bettany,Vision
8,Robert Downey Jr.,Sherlock Holmes
9,Robert Downey Jr.,Tony Stark


### Pregunta 2

La siguiente consulta natural es para cada actor, encontrar nombre de todas las peliculas dónde actuó este actor. Para esto necesitamos realizar un join entre las tres tablas:

In [ ]:
%%sql

SELECT Actores.anombre, Peliculas.pnombre
FROM Actores, actuo_en, Peliculas
WHERE Actores.aid = actuo_en.aid AND actuo_en.pid = Peliculas.pid
ORDER BY Actores.anombre, Peliculas.pnombre

 * postgresql+psycopg2://@/postgres


,anombre,pnombre
0,Anne Hathaway,Batman: The Dark Knight Rises
1,Anne Hathaway,Interstellar
2,Benedict Cumberbatch,Doctor Strange
3,Benedict Cumberbatch,Doctor Strange
4,Chris Evans,Avengers: Age of Ultron
5,Chris Evans,Avengers:Endgame
6,Chris Evans,Captain America: Civil War
7,Christian Bale,Batman: The Dark Knight
8,Christian Bale,Batman: The Dark Knight Rises
9,Paul Bettany,Avengers: Age of Ultron


Los duplicados aquí ocurren cuando un actor interpreta dos roles distintos en la misma película. Eliminando duplicados:

In [ ]:
%%sql

SELECT DISTINCT Actores.anombre, Peliculas.pnombre
FROM Actores, actuo_en, Peliculas
WHERE Actores.aid = actuo_en.aid AND actuo_en.pid = Peliculas.pid
ORDER BY Actores.anombre, Peliculas.pnombre

 * postgresql+psycopg2://@/postgres


,anombre,pnombre
0,Anne Hathaway,Batman: The Dark Knight Rises
1,Anne Hathaway,Interstellar
2,Benedict Cumberbatch,Doctor Strange
3,Chris Evans,Avengers: Age of Ultron
4,Chris Evans,Avengers:Endgame
5,Chris Evans,Captain America: Civil War
6,Christian Bale,Batman: The Dark Knight
7,Christian Bale,Batman: The Dark Knight Rises
8,Paul Bettany,Avengers: Age of Ultron
9,Paul Bettany,Iron Man


### Pregunta 3

Uno puede combinar joins con condiciones al mismo tiempo. Por ejemplo, si nos interesan nombres de los personajes que aparecen en una película estrenada después del año 2015, y con una calificación mayor a 8.5, necesitamos hacer:

- join entre `Peliculas` y `actuo_en`; y
- filtrar las filas que no corresponden a una película estrenada después del año 2015, y con una calificación menor a 8.5.

Lo debemos hacer así:

In [ ]:
%%sql

SELECT DISTINCT actuo_en.rol
FROM actuo_en, Peliculas
WHERE actuo_en.pid = Peliculas.pid AND Peliculas.paño >= 2015 AND Peliculas.pcalificacion >= 8.5

 * postgresql+psycopg2://@/postgres


,rol
0,Doctor Strange
1,Natasha Romanoff
2,Steve Rogers
3,Dormammu
4,Tony Stark


### Pregunta 4

Si nos interesan los roles y las películas del actor `Paul Bettany`, ordenadas por calificación que obtuvieron, de nuevo debemos hacer el join entre las tres relaciones:

In [ ]:
%%sql

SELECT Peliculas.pnombre, actuo_en.rol, Peliculas.pcalificacion
FROM Actores, actuo_en, Peliculas
WHERE Actores.anombre = 'Paul Bettany' AND Actores.aid = actuo_en.aid AND actuo_en.pid = Peliculas.pid
ORDER BY Peliculas.pcalificacion

 * postgresql+psycopg2://@/postgres


,pnombre,rol,pcalificacion
0,Avengers: Age of Ultron,J.A.R.V.I.S.,8.3
1,Avengers: Age of Ultron,Vision,8.3
2,Iron Man,J.A.R.V.I.S.,9.1


### Pregunta 5

Cómo podemos ver, `Paul Bettany` interpreta dos roles distintos en una misma película. Ahora nos interesa conseguir el `aid` de cada actor que interpreta dos roles distintos en una misma película. Para hacer esto, necesitamos hacer un join de la tabla `actuo_en` consigo misma. Esto se llama un *self-join*, y corresponde a armar un producto cruz de la tabla con su copia, y después filtrar algunas filas. En nuestro caso escribiremos:

In [ ]:
%%sql

SELECT DISTINCT A1.aid, A2.aid
FROM actuo_en AS A1, actuo_en AS A2
WHERE A1.aid = A2.aid AND A1.pid = A2.pid AND A1.rol <> A2.rol

 * postgresql+psycopg2://@/postgres


,aid,aid
0,6,6
1,7,7


Notense que aquí validamos dos cosas:

- que `aid`s coninciden (mismo actor)
- que `pid`s coinciden (misma película)
- que los roles son distintos

Para no repetir resultados (si (x,y) cumple con la condición, también lo hace (y,x)), ocupamos `DISTINCT`.

Adicionalmente, si también queremos recuperar el nombre del actor, podemos hacer un join con la tabla `Actores`:

In [ ]:
%%sql

SELECT DISTINCT Actores.anombre
FROM actuo_en AS A1, actuo_en AS A2, Actores
WHERE Actores.aid = A1.aid AND A1.aid = A2.aid AND A1.pid = A2.pid AND A1.rol <> A2.rol

 * postgresql+psycopg2://@/postgres


,anombre
0,Benedict Cumberbatch
1,Paul Bettany


Un self-join se puede repetir varias veces. Por ejemplo, para detectar actores que interpretaron tres roles en una misma película, podemos ocupar:

In [ ]:
%%sql

SELECT DISTINCT Actores.anombre
FROM actuo_en AS A1, actuo_en AS A2, actuo_en AS A3, Actores
WHERE Actores.aid = A1.aid AND 
      A1.aid = A2.aid AND A1.pid = A2.pid AND A1.rol <> A2.rol AND
      A1.aid = A3.aid AND A1.pid = A3.pid AND A1.rol <> A3.rol AND
      A2.aid = A3.aid AND A2.pid = A3.pid AND A2.rol <> A3.rol

 * postgresql+psycopg2://@/postgres


""


### Pregunta 6

Aquí `A1,A2`, y `A3` simulan los roles distintos en la misma película para el mismo actor. En nuestra base de datos no existe ninguno.

Finalmente, combinando joins con otros operadores podemos expresar algunas propiedades interesantes. Por ejemplo, podemos detectar todas las películas qué *no* tienen la mayor calificación. Esto podemos hacer a través de un self join de la tabla `Peliculas` consigo misma, quedando con las películas para cuales existe una película con mayor calificación:

In [ ]:
%%sql 

SELECT DISTINCT P1.pnombre
FROM Peliculas AS P1, Peliculas AS P2
WHERE P1.pid <> P2.pid AND P1.pcalificacion < P2.pcalificacion

 * postgresql+psycopg2://@/postgres


,pnombre
0,Doctor Strange
1,Sherlock Holmes
2,Batman: The Dark Knight Rises
3,Interstellar
4,Avengers: Age of Ultron
5,Captain America: Civil War
6,Avengers:Endgame
7,Batman: The Dark Knight


Si combinamos estos resultados con `EXCEPT`, también podemos encontrar la película con la mayor calificación:


In [ ]:
%%sql 

SELECT DISTINCT Peliculas.pnombre
FROM Peliculas
EXCEPT
SELECT DISTINCT P1.pnombre
FROM Peliculas AS P1, Peliculas AS P2
WHERE P1.pid <> P2.pid AND P1.pcalificacion < P2.pcalificacion

 * postgresql+psycopg2://@/postgres


,pnombre
0,Iron Man


### Resumen

En este tutorial aprendimos cómo analizar una base de datos de películas ocupando joins, y otros operadores de SQL. Adicionalmente, en el tutorial aprendimos el concepto de self-join, que puede tener mucha utilidad en ciertos casos.